In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mxnet
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers==3
!pip install torch
!pip install konlpy
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
import re
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

torch.manual_seed(615)
np.random.seed(615)

device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
import pandas as pd
import numpy as np
import re
import os
from konlpy.tag import Okt

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rc('font', family='Malgun Gothic')
%matplotlib inline

In [ ]:
import pandas as pd
path = '/content/drive/MyDrive/natural/'
train=pd.read_csv(path+'train.csv')
test=pd.read_csv(path+'test.csv')

In [ ]:
train=train[['사업명', '과제명', '요약문_연구목표', '요약문_연구내용', '요약문_한글키워드', '요약문_기대효과', 'label']]
test=test[['사업명', '과제명', '요약문_연구목표', '요약문_연구내용', '요약문_기대효과', '요약문_한글키워드']]
train['요약문_연구목표'].fillna('NAN', inplace=True)
test['요약문_연구목표'].fillna('NAN', inplace=True)
train['요약문_연구내용'].fillna('NAN', inplace=True)
test['요약문_연구내용'].fillna('NAN', inplace=True)
train['요약문_기대효과'].fillna('NAN', inplace=True)
test['요약문_기대효과'].fillna('NAN', inplace=True)
train['요약문_한글키워드'].fillna('NAN', inplace=True)
test['요약문_한글키워드'].fillna('NAN', inplace=True)

train['data']=train['사업명'] + train['과제명'] + train['요약문_연구목표'] + train['요약문_연구내용'] + train['요약문_기대효과'] + train['요약문_한글키워드']
test['data']=test['사업명'] + test['과제명'] + test['요약문_연구목표'] + test['요약문_연구내용'] + test['요약문_기대효과'] + test['요약문_한글키워드']

def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    sent_clean = re.sub(' +', ' ', sent_clean) 
    return sent_clean

In [ ]:
from sklearn.model_selection import train_test_split
trains, tests = train_test_split(train, test_size=0.2, random_state=615)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([clean_text(i)]) for i in dataset['data']]
        self.labels = [np.int32(i) for i in dataset['label']]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 120
batch_size = 120
warmup_ratio = 0.1
num_epochs = 46
max_grad_norm = 1
log_interval = 200
learning_rate = 3e-5

In [ ]:
data_train = BERTDataset(trains, 3, 2, tok, max_len, True, False)
data_test = BERTDataset(tests, 3, 2, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 46, 
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)


no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]


optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
highest_acc = 0
patience = 0

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) 
        optimizer.step()
        scheduler.step() 
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    
    model.eval()
    
    for test_batch_id, (test_token_ids, test_valid_length, test_segment_ids, test_label) in enumerate(tqdm_notebook(test_dataloader)):
        test_token_ids = test_token_ids.long().to(device)
        test_segment_ids = test_segment_ids.long().to(device)
        test_valid_length= test_valid_length
        test_label = test_label.long().to(device)
        test_out = model(token_ids, valid_length, segment_ids)
        test_loss = loss_fn(out, label)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (test_batch_id+1)))

    if test_acc > highest_acc:
        torch.save({
            'epoch': e,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': test_loss,
            }, '/content/drive/MyDrive/natural/torchckpt/model.pt')
        patience = 0
    else:
        print("test acc did not improved. best:{} current:{}".format(highest_acc, test_acc))
        patience += 1
        if patience > 5:
            break
    print('current patience: {}'.format(patience))
    print("************************************************************************************")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/1163 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 3.730957269668579 train acc 0.03333333333333333
epoch 1 batch id 201 loss 2.5041677951812744 train acc 0.4369402985074626
epoch 1 batch id 401 loss 1.5741596221923828 train acc 0.6256234413965086
epoch 1 batch id 601 loss 1.2751870155334473 train acc 0.68991957848031
epoch 1 batch id 801 loss 0.9575108289718628 train acc 0.7220661672908861
epoch 1 batch id 1001 loss 1.067197322845459 train acc 0.7411338661338661
epoch 1 train acc 0.7533247348810549


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/291 [00:00<?, ?it/s]

...

epoch 45 test acc 1.0
current patience: 0
************************************************************************************


  0%|          | 0/1163 [00:00<?, ?it/s]

epoch 46 batch id 1 loss 0.007861709222197533 train acc 0.9916666666666667
epoch 46 batch id 201 loss 0.004305522423237562 train acc 0.9963515754560527
epoch 46 batch id 401 loss 0.034994784742593765 train acc 0.9964256026600176
epoch 46 batch id 601 loss 0.02057291753590107 train acc 0.9964642262895191
epoch 46 batch id 801 loss 0.00023636898549739271 train acc 0.9964315439034561
epoch 46 batch id 1001 loss 0.014523906633257866 train acc 0.9964119214119238
epoch 46 train acc 0.9964817999426758


  0%|          | 0/291 [00:00<?, ?it/s]

epoch 46 test acc 1.0
current patience: 0
************************************************************************************


In [ ]:
class BERTDataset1(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([clean_text(i)]) for i in dataset['data']]
        self.labels = [np.int32(0) for i in dataset['data']]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
        #return (self.sentences[i] + (0, ))

    def __len__(self):
        return (len(self.sentences))


test_set = BERTDataset1(test, 3, 2, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=2)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

checkpoint = torch.load('/content/drive/MyDrive/natural/torchckpt/model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [ ]:
model.eval()


result = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  label = label.long().to(device)
  out = model(token_ids, valid_length, segment_ids)
  for i in out:
      logits = i
      logits = logits.detach().cpu().numpy()
      final = np.argmax(logits)
  result.append(final)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/43576 [00:00<?, ?it/s]

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/natural/sample_submission.csv')

submission['label'] = np.array(result)

submission.to_csv('/content/drive/MyDrive/natural/kobert_clean_baseline.csv', index=False)